<a href="https://colab.research.google.com/github/Sankoku/Human-Machine/blob/main/lab7_LM_Liashenko_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!wget https://github.com/BobAdamsEE/SouthParkData/raw/refs/heads/master/All-seasons.csv


--2025-06-08 16:01:29--  https://github.com/BobAdamsEE/SouthParkData/raw/refs/heads/master/All-seasons.csv
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/BobAdamsEE/SouthParkData/refs/heads/master/All-seasons.csv [following]
--2025-06-08 16:01:30--  https://raw.githubusercontent.com/BobAdamsEE/SouthParkData/refs/heads/master/All-seasons.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5533363 (5.3M) [text/plain]
Saving to: ‘All-seasons.csv.4’

All-seasons.csv.4   100%[===================>]   5.28M  --.-KB/s    in 0.1s    

2025-06-08 16:01:30 (47.6 MB/s) - ‘All-seasons.csv.4’ saved [5533363/

In [15]:
import nltk
import re
import numpy as np
from collections import Counter

nltk.download('punkt')
nltk.download('punkt_tab')

def load_data(file_path):
    """
    Завантажує дані з файлу.

    Args:
        file_path: шлях до файлу

    Returns:
        Рядок з текстом
    """
    with open(file_path, "r", encoding="utf-8") as f:
        data = f.read()
    return data

def split_to_sentences(data):
    """
    Розділяє текст на речення за розривом рядка '\n'.

    Args:
        data: текстовий рядок

    Returns:
        Список речень
    """
    sentences = data.split('\n')
    sentences = [s.strip() for s in sentences]
    sentences = [s for s in sentences if len(s) > 0]
    return sentences

def tokenize_sentences(sentences):
    """
    Токенізує речення на слова.

    Args:
        sentences: список речень

    Returns:
        Список списків токенів
    """
    tokenized_sentences = []
    for sentence in sentences:
        sentence = sentence.lower()
        tokenized = nltk.word_tokenize(sentence)
        tokenized_sentences.append(tokenized)
    return tokenized_sentences

def count_words(tokenized_sentences):
    """
    Підраховує частоту кожного слова в корпусі.

    Args:
        tokenized_sentences: список списків токенів

    Returns:
        Словник {слово: частота}
    """
    word_counts = {}
    for sentence in tokenized_sentences:
        for token in sentence:
            if token in word_counts:
                word_counts[token] += 1
            else:
                word_counts[token] = 1
    return word_counts

def get_words_with_nplus_frequency(tokenized_sentences, count_threshold):
    """
    Отримує слова, які з'являються не менше count_threshold разів.

    Args:
        tokenized_sentences: список списків токенів
        count_threshold: мінімальна частота

    Returns:
        Список слів, які з'являються не менше count_threshold разів
    """
    word_counts = count_words(tokenized_sentences)
    closed_vocab = [word for word, count in word_counts.items() if count >= count_threshold]
    return closed_vocab

def replace_oov_words_by_unk(tokenized_sentences, vocabulary, unknown_token="<unk>"):
    """
    Замінює слова, які не входять до словника, на токен unknown_token.

    Args:
        tokenized_sentences: список списків токенів
        vocabulary: список слів у словнику
        unknown_token: токен для невідомих слів

    Returns:
        Список списків токенів з заміною невідомих слів
    """
    vocabulary = set(vocabulary)
    replaced_tokenized_sentences = []

    for sentence in tokenized_sentences:
        replaced_sentence = []
        for token in sentence:
            if token in vocabulary:
                replaced_sentence.append(token)
            else:
                replaced_sentence.append(unknown_token)
        replaced_tokenized_sentences.append(replaced_sentence)

    return replaced_tokenized_sentences

def preprocess_data(train_data, test_data, count_threshold):
    """
    Попередня обробка даних.

    Args:
        train_data: тренувальні дані
        test_data: тестові дані
        count_threshold: мінімальна частота для слів

    Returns:
        Оброблені тренувальні та тестові дані, словник
    """
    vocabulary = get_words_with_nplus_frequency(train_data, count_threshold)
    train_data_replaced = replace_oov_words_by_unk(train_data, vocabulary)
    test_data_replaced = replace_oov_words_by_unk(test_data, vocabulary)

    return train_data_replaced, test_data_replaced, vocabulary


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [16]:
!pip install pandas

In [17]:
import pandas as pd

def load_southpark_data(file_path):
    """
    Завантажує дані South Park з CSV файлу.

    Args:
        file_path: шлях до CSV файлу

    Returns:
        Рядок з об'єднаним текстом усіх діалогів
    """
    # Завантажуємо CSV файл
    df = pd.read_csv(file_path)

    # Перевіряємо структуру даних
    print("Колонки в датасеті:", df.columns.tolist())
    print("Перші кілька рядків:")
    print(df.head())

    # Зазвичай у South Park датасетах є колонка з текстом діалогів
    # Потрібно знайти правильну назву колонки
    text_columns = [col for col in df.columns if 'text' in col.lower() or 'line' in col.lower() or 'dialogue' in col.lower()]

    if text_columns:
        text_column = text_columns[0]
        print(f"Використовуємо колонку: {text_column}")

        # Об'єднуємо всі діалоги в один текст
        all_text = '\n'.join(df[text_column].dropna().astype(str))
        return all_text
    else:
        # Якщо не знайшли підходящу колонку, виводимо всі колонки для ручного вибору
        print("Автоматично не вдалося визначити колонку з текстом.")
        print("Доступні колонки:", df.columns.tolist())
        return None


In [18]:
# Функція для запуску системи з South Park даними
def run_southpark_autocomplete():
    print("Завантаження даних South Park...")

    # Спочатку завантажуємо і перевіряємо структуру
    southpark_text = load_southpark_data("/content/All-seasons.csv")

    if southpark_text is None:
        print("Не вдалося завантажити дані. Перевірте структуру CSV файлу.")
        return

    print(f"Завантажено {len(southpark_text)} символів тексту")
    print("Перші 500 символів:")
    print(southpark_text[:500])

    # Тепер запускаємо систему автодоповнення
    print("\nТокенізація даних...")
    tokenized_data = get_tokenized_data(southpark_text)

    # Розділяємо дані на тренувальну та тестову вибірки
    import random
    random.seed(42)
    random.shuffle(tokenized_data)

    train_size = int(len(tokenized_data) * 0.8)
    train_data = tokenized_data[:train_size]
    test_data = tokenized_data[train_size:]

    print(f"Розмір тренувальної вибірки: {len(train_data)} речень")
    print(f"Розмір тестової вибірки: {len(test_data)} речень")

    print("Попередня обробка даних...")
    count_threshold = 2
    train_data_processed, test_data_processed, vocabulary = preprocess_data(train_data, test_data, count_threshold)

    print(f"Розмір словника: {len(vocabulary)} слів")
    print("Перші 20 слів зі словника:", vocabulary[:20])

    print("Запуск інтерфейсу користувача...")
    create_ui(train_data_processed, vocabulary)

In [19]:
def count_n_grams(data, n, start_token='<s>', end_token='</s>'):
    """
    Підраховує кількість n-грам у даних.

    Args:
        data: список списків токенів
        n: розмір n-грами
        start_token: токен початку речення
        end_token: токен кінця речення

    Returns:
        Словник {n-грама: кількість}
    """
    n_grams = {}

    for sentence in data:
        # Додаємо токени початку та кінця
        sentence = [start_token] * (n-1) + sentence + [end_token]
        sentence = tuple(sentence)

        # Формуємо n-грами
        for i in range(len(sentence) - n + 1):
            n_gram = sentence[i:i+n]

            if n_gram in n_grams:
                n_grams[n_gram] += 1
            else:
                n_grams[n_gram] = 1

    return n_grams


In [20]:
def estimate_probability(word, previous_n_gram, n_gram_counts, n_plus1_gram_counts, vocabulary_size, k=1.0):
    """
    Оцінює ймовірність слова word, враховуючи попередню n-граму.

    Args:
        word: слово для оцінки ймовірності
        previous_n_gram: попередня n-грама
        n_gram_counts: словник кількостей n-грам
        n_plus1_gram_counts: словник кількостей (n+1)-грам
        vocabulary_size: розмір словника
        k: параметр згладжування

    Returns:
        Ймовірність
    """
    # Перетворення списку на кортеж для використання як ключа словника
    previous_n_gram = tuple(previous_n_gram)

    # Якщо попередня n-грама є у словнику, взяти її кількість, інакше встановити 0
    previous_n_gram_count = n_gram_counts.get(previous_n_gram, 0)

    # Обчислення знаменника з використанням k-згладжування
    denominator = previous_n_gram_count + k * vocabulary_size

    # Формування (n+1)-грами додаванням слова до попередньої n-грами
    n_plus1_gram = previous_n_gram + (word,)

    # Якщо (n+1)-грама є у словнику, взяти її кількість, інакше встановити 0
    n_plus1_gram_count = n_plus1_gram_counts.get(n_plus1_gram, 0)

    # Обчислення чисельника із згладжуванням
    numerator = n_plus1_gram_count + k

    # Обчислення ймовірності
    probability = numerator / denominator

    return probability


In [21]:
def estimate_probabilities(previous_n_gram, n_gram_counts, n_plus1_gram_counts, vocabulary, k=1.0):
    """
    Оцінює ймовірності всіх можливих наступних слів.

    Args:
        previous_n_gram: попередня n-грама
        n_gram_counts: словник кількостей n-грам
        n_plus1_gram_counts: словник кількостей (n+1)-грам
        vocabulary: список слів у словнику
        k: параметр згладжування

    Returns:
        Словник {слово: ймовірність}
    """
    vocabulary = vocabulary + ['</s>', '<unk>']  # Додаємо спеціальні токени
    vocabulary_size = len(vocabulary)

    probabilities = {}
    for word in vocabulary:
        probability = estimate_probability(word, previous_n_gram,
                                           n_gram_counts, n_plus1_gram_counts,
                                           vocabulary_size, k)
        probabilities[word] = probability

    return probabilities


In [22]:
def calculate_perplexity(sentence, n_gram_counts, n_plus1_gram_counts, vocabulary_size, k=1.0):
    """
    Обчислює перплексію для речення.

    Args:
        sentence: список токенів
        n_gram_counts: словник кількостей n-грам
        n_plus1_gram_counts: словник кількостей (n+1)-грам
        vocabulary_size: розмір словника
        k: параметр згладжування

    Returns:
        Перплексія
    """
    # Довжина попередніх слів (n-1 для n-грам)
    n = len(list(n_gram_counts.keys())[0])

    # Додаємо токени початку та кінця
    sentence = ['<s>'] * n + sentence + ['</s>']
    sentence = tuple(sentence)

    # Загальна довжина речення (після додавання токенів)
    N = len(sentence)

    # Змінна для зберігання добутку
    product_pi = 1.0

    # Для кожного слова в реченні (починаючи з n-го)
    for t in range(n, N):
        # Отримуємо n-граму перед словом на позиції t
        n_gram = sentence[t-n:t]

        # Отримуємо слово на позиції t
        word = sentence[t]

        # Оцінюємо ймовірність слова, враховуючи n-граму
        probability = estimate_probability(word, n_gram,
                                           n_gram_counts, n_plus1_gram_counts,
                                           vocabulary_size, k)

        # Оновлюємо добуток
        product_pi *= 1/probability

    # Обчислюємо N-й корінь з добутку
    perplexity = product_pi**(1/N)

    return perplexity


In [23]:
def suggest_a_word(previous_tokens, n_gram_counts, n_plus1_gram_counts, vocabulary, k=1.0, start_with=None):
    """
    Пропонує наступне слово на основі попередніх токенів.

    Args:
        previous_tokens: попередні токени
        n_gram_counts: словник кількостей n-грам
        n_plus1_gram_counts: словник кількостей (n+1)-грам
        vocabulary: список слів у словнику
        k: параметр згладжування
        start_with: початкові літери для фільтрації (опціонально)

    Returns:
        Кортеж (найімовірніше слово, його ймовірність)
    """
    # Довжина попередніх слів для n-грам
    n = len(list(n_gram_counts.keys())[0])

    # Додаємо токени початку речення
    previous_tokens = ['<s>'] * n + previous_tokens

    # Отримуємо останні n слів як попередню n-граму
    previous_n_gram = previous_tokens[-n:]

    # Обчислюємо ймовірності для всіх можливих наступних слів
    probabilities = estimate_probabilities(previous_n_gram,
                                           n_gram_counts, n_plus1_gram_counts,
                                           vocabulary, k)

    # Ініціалізуємо змінні для найімовірнішого слова
    suggestion = None
    max_prob = 0

    # Перебираємо всі слова та їх ймовірності
    for word, prob in probabilities.items():
        # Якщо вказані початкові літери для фільтрації
        if start_with:
            # Перевіряємо, чи слово починається з вказаних літер
            if not word.startswith(start_with):
                continue

        # Якщо ймовірність вища за поточну максимальну
        if prob > max_prob:
            suggestion = word
            max_prob = prob

    return suggestion, max_prob

def get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k=1.0, start_with=None, top_n=5):
    """
    Отримує список найімовірніших наступних слів від різних N-грамних моделей.

    Args:
        previous_tokens: попередні токени
        n_gram_counts_list: список словників кількостей n-грам
        vocabulary: список слів у словнику
        k: параметр згладжування
        start_with: початкові літери для фільтрації (опціонально)
        top_n: кількість пропозицій

    Returns:
        Список пропозицій
    """
    suggestions = []

    # Перебираємо всі N-грамні моделі
    for i in range(len(n_gram_counts_list) - 1):
        n_gram_counts = n_gram_counts_list[i]
        n_plus1_gram_counts = n_gram_counts_list[i+1]

        # Отримуємо пропозицію від кожної моделі
        all_probabilities = estimate_probabilities(previous_tokens[-i-1:] if i < len(previous_tokens) else ['<s>'] * (i+1-len(previous_tokens)) + previous_tokens,
                                                n_gram_counts, n_plus1_gram_counts,
                                                vocabulary, k)

        # Фільтруємо за початковими літерами, якщо вказано
        if start_with:
            filtered_probabilities = {word: prob for word, prob in all_probabilities.items()
                                     if word.startswith(start_with)}
        else:
            filtered_probabilities = all_probabilities

        # Сортуємо за ймовірністю та вибираємо top_n найкращих
        top_suggestions = sorted(filtered_probabilities.items(),
                               key=lambda x: x[1], reverse=True)[:top_n]

        suggestions.append(top_suggestions)

    return suggestions


In [24]:
import ipywidgets as widgets
from IPython.display import display, clear_output

def create_ui(train_data_processed, vocabulary):
    """
    Створює інтерфейс користувача для системи автозавершення.

    Args:
        train_data_processed: оброблені тренувальні дані
        vocabulary: словник
    """
    # Обчислюємо N-грами різних розмірів
    n_gram_counts_list = []
    for n in range(1, 6):  # Від уніграм до 5-грам
        n_gram_counts = count_n_grams(train_data_processed, n)
        n_gram_counts_list.append(n_gram_counts)

    # Створюємо віджети
    text_input = widgets.Text(
        value='',
        placeholder='Введіть текст...',
        description='Текст:',
        disabled=False,
        layout=widgets.Layout(width='500px')
    )

    prefix_input = widgets.Text(
        value='',
        placeholder='Обмеження за першими літерами (необов\'язково)',
        description='Початок:',
        disabled=False,
        layout=widgets.Layout(width='500px')
    )

    n_gram_dropdown = widgets.Dropdown(
        options=[('Уніграми', 1), ('Біграми', 2), ('Триграми', 3), ('4-грами', 4), ('5-грами', 5)],
        value=2,
        description='N-грами:',
        disabled=False,
    )

    k_slider = widgets.FloatSlider(
        value=1.0,
        min=0.01,
        max=5.0,
        step=0.01,
        description='K:',
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True,
        readout_format='.2f',
    )

    suggestion_output = widgets.Output()

    def on_button_click(suggestion):
        """Функція, що виконується при натисканні на пропозицію"""
        current_text = text_input.value
        # Додаємо пробіл, якщо потрібно
        if current_text and not current_text.endswith(' '):
            current_text += ' '
        # Додаємо запропоноване слово
        text_input.value = current_text + suggestion + ' '

    def update_suggestions(_):
        """Оновлює пропозиції на основі введеного тексту"""
        with suggestion_output:
            clear_output()

            # Отримуємо поточний текст
            current_text = text_input.value.strip()
            if not current_text:
                print("Введіть текст для отримання пропозицій.")
                return

            # Токенізуємо текст
            tokens = current_text.lower().split()

            # Отримуємо N-грамну модель
            n = n_gram_dropdown.value
            if n > len(n_gram_counts_list):
                print(f"Недостатньо даних для {n}-грам.")
                return

            # Отримуємо початкові літери для фільтрації
            start_with = prefix_input.value if prefix_input.value else None

            # Отримуємо пропозиції
            if n == 1:  # Для уніграм
                probabilities = estimate_probabilities([],
                                                      n_gram_counts_list[0],
                                                      n_gram_counts_list[1],
                                                      vocabulary,
                                                      k=k_slider.value)
            else:  # Для N-грам, де N > 1
                previous_tokens = tokens[-(n-1):] if len(tokens) >= n-1 else ['<s>'] * (n-1-len(tokens)) + tokens
                probabilities = estimate_probabilities(previous_tokens,
                                                      n_gram_counts_list[n-2],
                                                      n_gram_counts_list[n-1],
                                                      vocabulary,
                                                      k=k_slider.value)

            # Фільтруємо за початковими літерами, якщо вказано
            if start_with:
                filtered_probabilities = {word: prob for word, prob in probabilities.items()
                                         if word.startswith(start_with) and word not in ['<s>', '</s>', '<unk>']}
            else:
                filtered_probabilities = {word: prob for word, prob in probabilities.items()
                                         if word not in ['<s>', '</s>', '<unk>']}

            # Отримуємо топ-5 пропозицій
            top_suggestions = sorted(filtered_probabilities.items(),
                                   key=lambda x: x[1], reverse=True)[:5]

            if not top_suggestions:
                print("Немає пропозицій для даного контексту.")
                return

            print(f"Пропозиції для '{current_text}':")
            suggestion_buttons = []
            for word, prob in top_suggestions:
                button = widgets.Button(
                    description=f"{word} ({prob:.4f})",
                    button_style='',
                    tooltip=f'Ймовірність: {prob:.4f}'
                )
                button.on_click(lambda b, word=word: on_button_click(word))
                suggestion_buttons.append(button)

            display(widgets.HBox(suggestion_buttons))

    # Створюємо кнопку для отримання пропозицій
    suggest_button = widgets.Button(
        description='Запропонувати',
        disabled=False,
        button_style='',
        tooltip='Натисніть для отримання пропозицій'
    )
    suggest_button.on_click(update_suggestions)

    # Відображаємо інтерфейс
    display(text_input)
    display(prefix_input)
    display(widgets.HBox([n_gram_dropdown, k_slider]))
    display(suggest_button)
    display(suggestion_output)

# Приклад використання:
# create_ui(train_data_processed, vocabulary)


In [25]:
def get_tokenized_data(data):
    """
    Make a list of tokenized sentences

    Args:
        data: String

    Returns:
        List of lists of tokens
    """
    # Get the sentences by splitting up the data
    sentences = split_to_sentences(data)

    # Get the list of lists of tokens by tokenizing the sentences
    tokenized_sentences = tokenize_sentences(sentences)

    return tokenized_sentences


# Основна функція для запуску системи
def run_autocomplete_system(file_path):
    print("Завантаження даних...")
    data = load_data(file_path)

    print("Токенізація даних...")
    tokenized_data = get_tokenized_data(data)

    # Розділяємо дані на тренувальну та тестову вибірки (80% / 20%)
    import random
    random.seed(42)
    random.shuffle(tokenized_data)

    train_size = int(len(tokenized_data) * 0.8)
    train_data = tokenized_data[:train_size]
    test_data = tokenized_data[train_size:]

    print(f"Розмір тренувальної вибірки: {len(train_data)} речень")
    print(f"Розмір тестової вибірки: {len(test_data)} речень")

    print("Попередня обробка даних...")
    count_threshold = 2  # Мінімальна частота слова
    train_data_processed, test_data_processed, vocabulary = preprocess_data(train_data, test_data, count_threshold)

    print(f"Розмір словника: {len(vocabulary)} слів")

    print("Запуск інтерфейсу користувача...")
    create_ui(train_data_processed, vocabulary)

# Приклад запуску системи:
# run_autocomplete_system("your_corpus.txt")


In [26]:
run_southpark_autocomplete()

Завантаження даних South Park...
Колонки в датасеті: ['Season', 'Episode', 'Character', 'Line']
Перші кілька рядків:
  Season Episode Character                                               Line
0     10       1      Stan         You guys, you guys! Chef is going away. \n
1     10       1      Kyle                        Going away? For how long?\n
2     10       1      Stan                                         Forever.\n
3     10       1      Chef                                  I'm sorry boys.\n
4     10       1      Stan  Chef said he's been bored, so he joining a gro...
Використовуємо колонку: Line
Завантажено 4493489 символів тексту
Перші 500 символів:
You guys, you guys! Chef is going away. 

Going away? For how long?

Forever.

I'm sorry boys.

Chef said he's been bored, so he joining a group called the Super Adventure Club. 

Wow!

Chef?? What kind of questions do you think adventuring around the world is gonna answer?!

What's the meaning of life? Why are we here?

I hope 

Text(value='', description='Текст:', layout=Layout(width='500px'), placeholder='Введіть текст...')

Text(value='', description='Початок:', layout=Layout(width='500px'), placeholder="Обмеження за першими літерам…

Button(description='Запропонувати', style=ButtonStyle(), tooltip='Натисніть для отримання пропозицій')

Output()

In [27]:
from IPython.display import Javascript
Javascript('IPython.notebook.metadata.widgets = {}')

<IPython.core.display.Javascript object>